In [ ]:
import numpy as np
from collections import defaultdict
import json

In [ ]:
# all this +/- stuff is only really used for the 'current year MOV' prediction
iv = np.array([71.3 ,  6.65, 17.4 ])

def get_mov(x):
    offset = len(iv)//3
    tot = -1.977479789
    for i in range(offset):
        tot += (iv[offset+i])*(np.tanh((x-iv[i])/iv[2*offset+i])+1.0)
    return tot

# estimated mov per ovr
o2m = {}
for i in np.arange(101):
    o2m[i] = get_mov(i)

# replacement level
r_lvl = -1.425


age_shift ={19: 6.4,
 20: 6.7,
 21: 5.0,
 22: 4.4,
 23: 2.2,
 24: 2.0,
 25: 0.1,
 26: -0.1,
 27: -1.2,
 28: -1.4,
 29: -2.0,
 30: -2.7,
 31: -3.6,
 32: -3.7,
 33: -3.7,
 34: -4.7,
 35: -5.1}

max_shift = min(age_shift.values())

# replacement level
RL = 45.3

# expected mov, weight for players, weight for salary cap space
weights = {
0 : ( [1.312, 1.025] , 1.26 ),
1 : ( [11.141, 0.725] , 1.79 ),
2 : ( [9.143, 0.462] , 2.76 ),
}

# draft value
draftP = np.array([0.33331122, 0.30847174, 0.58789643])

# team value
team_mov = [-0.17330874,  0.33627302, 99.72968488]


In [ ]:
team = { 'mov': -4, 
        'p' : [(22,66,22000,2),(22,66,22000,2),(22,66,22000,2),(22,66,22000,2)], #(age,ovr,salary,years_left)
         'd' : [(1, 0, 8.6),(1, 0, -5.35),(2,0,3),(1,1,3),(2,1,3),(1,2,3),(2,2,3),(1,3,3),(2,3,3),(1,4,3),(2,4,3)] #(round,years_left,team_MOV)
       }
def get_team_value(team,sCap=90000,minS=750):
    YEARS_TO_MODEL = 3
    teamNum = 30
    CON_BUMP = 0.25
    
    # turn mov into draft pick and future mov
    m2pos = lambda x: int(round(np.clip( (teamNum-1)/(1+np.exp(0.0036 - 0.368*(x))),0,teamNum-1)))
    m2next = lambda year,mov: [1,0.5,0.25,0.08,0.03,0.01,0.01,0.01,0.01,0.01][year]*mov 

    # turn draft picks into specific predictions
    draft_picks = [ (yr,m2pos(m2next(yr,mov))+teamNum*(rnd-1)) for rnd,yr,mov in team['d']]
    
    pars = defaultdict(list)
    tss = defaultdict(int)
    dpars = []

    dpars = []
    for i in range(YEARS_TO_MODEL):
        for age,ovr,con,yrl in team['p']:
            tss[i] += con
            ovr2 = ovr
            for j in range(i):
                ovr2+=age_shift.get(age+j,max_shift)
            pars[i].append(ovr2)
    for age,ovr,con,yrl in team['p']:
        if yrl > YEARS_TO_MODEL:
            extra_years = yrl - YEARS_TO_MODEL
            amount_to_add = CON_BUMP*(con * extra_years)/YEARS_TO_MODEL
            for i  in range(YEARS_TO_MODEL):
                tss[i] += amount_to_add

    for yr,p in draft_picks:
        dpars.append((0.99**yr)*draftP[1]*np.exp(-draftP[0]*p**draftP[2]))
    
    pred_movs = []
    for i in range(YEARS_TO_MODEL):
        play = [p for p in pars[i] if p >= RL]
        lp = len(play)
        if lp < 10:
            play= play + (10-lp)*[RL]
        play = sorted(play,reverse=True)[:10]
        play_s = sum([np.exp(i*team_mov[0])*p for i,p in enumerate(play)])*team_mov[1] -team_mov[2]

        cap_hit = tss[i] + (10-lp)*minS 

        diff = (sCap-cap_hit)/sCap
        cap_space = np.maximum(diff,0.1*diff)
        x = weights.get(i,weights[2])

        p_mov = x[1] * (x[0][0] * cap_space + x[0][1] * play_s)
        
        pred_movs.append(p_mov)
        
    cA,cB = (0.42422076396359476, -3.94406073999211)

    win_p = [1.0/(1+np.exp(-mov*cA -cB)) for mov in pred_movs]

    # discount factor for the future, more uncertainty, less sure reward
    value = [wp*(0.95**(i)) for i,wp in enumerate(win_p)]
    #print(sum(value),sum(dpars))
    return sum(value) + sum(dpars)
get_team_value(team,90000,750)

In [ ]:
data = json.load(open('real_2020.json','rt',encoding='utf-8-sig'))
gA = {_['key']:_['value'] for _ in data['gameAttributes']}

In [ ]:
team_players = defaultdict(list)
team_names = {}
season = gA['season']
sCap = gA['salaryCap']
minS = gA['minContract']
maxS = gA['maxContract']

team_injuries = defaultdict(list)
for p in data['players']:
    if p['tid'] >= 0:
        ovr = p['ratings'][-1]['ovr']
        age = season - p['born']['year']
        salary = p['contract']['amount']
        years_left = p['contract']['exp']-season
        team_players[p['tid']].append((age,ovr,salary,years_left))
        team_injuries[p['tid']].append(p['injury']['gamesRemaining'])


In [ ]:
team_movs = defaultdict(float)

for t in data['teams']:
    tid = t['tid']
    team_names[t['tid']] = t['abbrev']
    for ts in t['stats']:
        if ts['playoffs']:
            continue
        current_mov = 0
        gp = ts['gp']+1e-9
        gl = 82-gp+1e-9
        if season == ts['season'] and not ts['playoffs'] and ts['gp']>0:
            mov = (ts['pts'] - ts['oppPts']) / ts['gp'];
            current_mov = mov
        estimated_mov = sum(sorted([o2m[_[1]]*(max(gl-i,0)/gl) for _,i in zip(team_players[tid],team_injuries[tid])])[-10:])
        team_movs[tid] = (gp/82)*current_mov + (gl/82)*estimated_mov    

In [ ]:
team_picks = defaultdict(list)
for d in data['draftPicks']:
    mov = team_movs[d['originalTid']]
    tid = d['tid']
    rnd = d['round']
    yl = d['season']-season
    team_picks[tid].append((rnd,yl,mov))

In [ ]:
teams_vals = []
for i in range(len(data['teams'])):
    #print(team_names[i])
    val = get_team_value({'mov':team_movs[i],'p':team_players[i],'d':team_picks[i]},sCap,minS)
    teams_vals.append((val,team_names[i]))

In [ ]:
for v,t in sorted(teams_vals)[::-1]:
    print(round(100*v,2),t)

In [ ]:
import random
deals = []
for i in range(1000):
    t1 = np.random.randint(30)
    t2 = np.random.randint(30)
    if t1 == t2:
        continue
    
    t1vo = get_team_value({'mov':team_movs[t1], 'p':team_players[t1],'d':team_picks[t1]},sCap,minS)
    t2vo = get_team_value({'mov':team_movs[t2], 'p':team_players[t2],'d':team_picks[t2]},sCap,minS)
    pn = np.random.randint(len(team_picks[t1]))
    picks = [_ for _ in team_picks[t1]]
    pick = picks[pn]        
    del picks[pn]

    for pi in range(len(team_players[t2])):
        local_p = [_ for _ in team_players[t2]]
        player = local_p[pi]
        del local_p[pi]
        t1v = get_team_value({'mov':team_movs[t1],'p':team_players[t1] + [player],'d':picks},sCap,minS)
        t2v = get_team_value({'mov':team_movs[t2],'p':local_p,'d':team_picks[t2] + [pick]},sCap,minS)
        if t1v > t1vo and t2v > t2vo:
            val = min((t1v-t1vo),(t2v-t2vo))
            deals.append((val,team_names[t1],pick,team_names[t2],player))

In [ ]:
sorted(deals,reverse=True)

In [ ]:
import random
deals = []
for i in range(1000):
    t1 = np.random.randint(30)
    t2 = np.random.randint(30)
    if t1 == t2:
        continue
    
    t1vo = get_team_value({'mov':team_movs[t1],'p':team_players[t1],'d':team_picks[t1]},sCap,minS)
    t2vo = get_team_value({'mov':team_movs[t2],'p':team_players[t2],'d':team_picks[t2]},sCap,minS)
    
    pn = np.random.randint(len(team_players[t1]))
    players1 = [_ for _ in team_players[t1]]
    pick = players1[pn]        
    del players1[pn]

    for pi in range(len(team_players[t2])):
        local_p = [_ for _ in team_players[t2]]
        player = local_p[pi]
        del local_p[pi]
        t1v = get_team_value({'mov':team_movs[t1],'p':players1 + [player],'d':team_picks[t1]},sCap,minS)
        t2v = get_team_value({'mov':team_movs[t2],'p':local_p + [pick],'d':team_picks[t2]},sCap,minS)
        if t1v > t1vo and t2v > t2vo:
            val = (t1v-t1vo) + (t2v-t2vo)
            deals.append((val,team_names[t1],pick,team_names[t2],player))

In [ ]:
for d in sorted(deals,reverse=True):
    if d[4][2]/d[2][2] < 1.25 and d[4][2]/d[2][2] > 1/1.25:
        print(d)

In [ ]:
import random
deals = set()
for i in range(1000):
    t1 = np.random.randint(30)
    t2 = np.random.randint(30)
    if t1 == t2:
        continue
    
    t1vo = get_team_value({'mov':team_movs[t1],'p':team_players[t1],'d':team_picks[t1]},sCap,minS)
    t2vo = get_team_value({'mov':team_movs[t2],'p':team_players[t2],'d':team_picks[t2]},sCap,minS)
    
    pn = np.random.randint(len(team_players[t1]))
    players1 = [_ for _ in team_players[t1]]
    pick = players1[pn]        
    del players1[pn]

    for pi in range(len(team_players[t2])):
        local_p = [_ for _ in team_players[t2]]
        player = local_p[pi]
        del local_p[pi]
        if not (pick[2]/player[2] < 1.25 and pick[2]/player[2] > 1/1.25):
            continue
        for p2 in range(len(team_picks[t2])):
            local_picks = [_ for _ in team_picks[t2]]
            pick2 = local_picks[p2]       
            del local_picks[p2]
            t1v1 = get_team_value({'mov':team_movs[t1],'p':players1 + [player],'d':team_picks[t1]},sCap,minS)

            t1v = get_team_value({'mov':team_movs[t1],'p':players1 + [player],'d':team_picks[t1] + [pick2]},sCap,minS)
            t2v = get_team_value({'mov':team_movs[t2],'p':local_p + [pick],'d':local_picks },sCap,minS)
            if t1v1 < t1vo and t1v > t1vo and t2v > t2vo:
                v1 = (t1v-t1vo)
                v2 = (t2v-t2vo)
                val = min( v1 , v2)
                deals.add((val,v1,v2,team_names[t1],pick,team_names[t2],player,pick2))
                

In [ ]:
sorted(list(deals),reverse=True)

In [ ]:
team_names

In [ ]:
team_test

In [ ]:
team_test = {'mov':team_movs[7],'d':team_picks[7],'p':team_players[7]}

In [ ]:
get_team_value(team_test,sCap,minS)

In [ ]:
team_test2 = {'mov': -5.372719925920409,
 'd': [(1, 0, -5.372719925920409),
  (2, 0, -5.372719925920409),
  (2, 0, 9.030763602443432),
  (1, 1, -5.372719925920409),
  (2, 1, -4.958921830230271),
  (2, 1, 4.178927470632469),
  (1, 2, -5.372719925920409),
  (2, 2, -5.372719925920409),
  (2, 2, -4.958921830230271),
  (1, 3, -5.372719925920409),
  (2, 3, -5.372719925920409),
  (2, 3, -7.548780876962224),
  (1, 4, -5.372719925920409),
  (2, 4, -5.372719925920409),
  (1, 5, -5.372719925920409),
  (2, 5, -5.372719925920409),
  (2, 5, -0.5734780821022222),
  (1, 6, -5.372719925920409),
  (2, 6, -5.372719925920409),
  (2, 6, -4.91664629842538)],
 'p': [(31, 50, 13350, 1),
  (26, 58, 17500, 2),
  (25, 27, 1700, 0),
  (22, 31, 1550, 1),
  (23, 25, 950, 0),
  (21, 51, 4850, 2),
  (22, 56, 4250, 1),
  (22, 58, 6650, 1),
  (23, 52, 8050, 2),
  (23, 59, 4150, 0),]}
get_team_value(team_test2,sCap,minS)

In [ ]:
team_test2 = {'mov': -5.372719925920409,
 'd': [
  (2, 0, -5.372719925920409),
  (2, 0, 9.030763602443432),
  (1, 1, -5.372719925920409),
  (2, 1, -4.958921830230271),
  (2, 1, 4.178927470632469),
  (1, 2, -5.372719925920409),
  (2, 2, -5.372719925920409),
  (2, 2, -4.958921830230271),
  (1, 3, -5.372719925920409),
  (2, 3, -5.372719925920409),
  (2, 3, -7.548780876962224),
  (1, 4, -5.372719925920409),
  (2, 4, -5.372719925920409),
  (1, 5, -5.372719925920409),
  (2, 5, -5.372719925920409),
  (2, 5, -0.5734780821022222),
  (1, 6, -5.372719925920409),
  (2, 6, -5.372719925920409),
  (2, 6, -4.91664629842538)],
 'p': [(20,50,7500,3),
    (31, 50, 13350, 1),
  (26, 58, 17500, 2),
  (25, 27, 1700, 0),
  (22, 31, 1550, 1),
  (23, 25, 950, 0),
  (21, 51, 4850, 2),
  (22, 56, 4250, 1),
  (22, 58, 6650, 1),
  (23, 52, 8050, 2),
  (23, 59, 4150, 0),]}
get_team_value(team_test2,sCap,minS)

In [ ]:
team_test = {'mov':team_movs[2],'d':team_picks[2],'p':team_players[2]}
get_team_value(team_test,sCap,minS)

In [ ]:
team_test2 = {
    'mov':team_movs[2],
    'd': [(2, 0, 1.652969631948502),
 (2, 0, 0.05698951843920175),
 (2, 1, 4.407755127986256),
 (1, 1, -4.91664629842538),
 (1, 1, -3.306876503590236),
 (2, 1, -3.306876503590236),
 (1, 2, -4.91664629842538),
 (2, 2, -4.91664629842538),
 (2, 2, 6.825253437957853),
 (1, 3, -4.91664629842538),
 (1, 5, -4.91664629842538),
 (2, 5, -4.91664629842538),
 (1, 6, -4.91664629842538)],
 'p': [(24, 51, 4800, 2),
 (24, 55, 1650, 2),
 (25, 59, 29550, 3),
 (27, 22, 1650, 2),
 (21, 25, 3900, 3),
 (30, 59, 20200, 4),
 (30, 64, 38000, 4),
 (32, 68, 40250, 2),
 (26, 26, 1650, 2),
 (23, 46, 1800, 1),
 (27, 26, 100, 0),
 (20, 33, 1750, 3),
 (23, 38, 200, 0),
 (28, 38, 1850, 2),
 (23, 29, 1650, 2)]}
get_team_value(team_test2,sCap,minS)

In [ ]:
team_players[2]